<a href="https://colab.research.google.com/github/doronin99/RecoServiceTemplate/blob/task5/notebooks/recbole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dependencies installing

In [ ]:
pip install -q recbole

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.6 MB/s eta 0:00:00


In [ ]:
pip install -q ray

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 MB 12.0 MB/s eta 0:00:00


In [ ]:
pip install -q kmeans_pytorch

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import ast
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import time

from collections import Counter
from random import randint, random
from scipy.sparse import coo_matrix, hstack
from sklearn.metrics.pairwise import euclidean_distances, cosine_distances, cosine_similarity

import logging
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.sequential_recommender import GRU4Rec, Caser
from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger
from recbole.quick_start import run_recbole

## Data loading

In [ ]:
!wget -q https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip -O data_original.zip
!unzip -o data_original.zip
!rm data_original.zip

Archive:  data_original.zip
   creating: data_original/
  inflating: data_original/interactions.csv  
  inflating: __MACOSX/data_original/._interactions.csv  
  inflating: data_original/users.csv  
  inflating: __MACOSX/data_original/._users.csv  
  inflating: data_original/items.csv  
  inflating: __MACOSX/data_original/._items.csv  


In [ ]:
interactions_df = pd.read_csv('data_original/interactions.csv')
users_df = pd.read_csv('data_original/users.csv')
items_df = pd.read_csv('data_original/items.csv')

In [ ]:
interactions_df['t_dat'] = pd.to_datetime(interactions_df['last_watch_dt'], format="%Y-%m-%d")
interactions_df['timestamp'] = interactions_df.t_dat.values.astype(np.int64) // 10 ** 9

df = interactions_df[['user_id', 'item_id', 'timestamp']].rename(
    columns={'user_id': 'user_id:token', 'item_id': 'item_id:token', 'timestamp': 'timestamp:float'})

In [ ]:
!mkdir recbox_data

In [ ]:
df.to_csv('recbox_data/recbox_data.inter', index=False, sep='\t')

## Main

In [ ]:
parameter_dict = {
    'data_path': '',
    'USER_ID_FIELD': 'user_id',
    'ITEM_ID_FIELD': 'item_id',
    'TIME_FIELD': 'timestamp',
    'device': 'GPU',
    'user_inter_num_interval': "[40,inf)",
    'item_inter_num_interval': "[40,inf)",
    'load_col': {'inter': ['user_id', 'item_id', 'timestamp']},
    'neg_sampling': None,
    'epochs': 10,
    'eval_args': {
        'split': {'RS': [9, 0, 1]},
        'group_by': 'user',
        'order': 'TO',
        'mode': 'full'}
}
config = Config(model='MultiVAE', dataset='recbox_data', config_dict=parameter_dict)

# init random seed
init_seed(config['seed'], config['reproducibility'])

# logger initialization
init_logger(config)
logger = getLogger()
# Create handlers
c_handler = logging.StreamHandler()
c_handler.setLevel(logging.INFO)
logger.addHandler(c_handler)

# write config info into log
# logger.info(config)

In [ ]:
dataset = create_dataset(config)
logger.info(dataset)

In [ ]:
# dataset splitting
train_data, valid_data, test_data = data_preparation(config, dataset)

In [ ]:
%%time
model_list = ['ItemKNN', 'DMF', 'RecVAE', 'ConvNCF', 'LightGCN']

for model_name in model_list:
    print(f"running {model_name}...")
    start = time.time()
    result = run_recbole(model=model_name, dataset = 'recbox_data',config_dict = parameter_dict)
    t = time.time() - start
    print(f"It took {t/60:.2f} mins")
    print(result)

running ItemKNN...


command line args [-f /root/.local/share/jupyter/runtime/kernel-a7e19ae3-fb9e-4161-a147-e3d1f962669a.json] will not be used in RecBole
Evaluate   : 100%|███████████████████████████████████████████| 13354/13354 [00:28<00:00, 463.71it/s]


It took 3.38 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.0909), ('mrr@10', 0.1768), ('ndcg@10', 0.088), ('hit@10', 0.3654), ('precision@10', 0.0504)])}
running DMF...


command line args [-f /root/.local/share/jupyter/runtime/kernel-a7e19ae3-fb9e-4161-a147-e3d1f962669a.json] will not be used in RecBole
Max value of item's history interaction records has reached 44.36540621490079% of the total.
Max value of user's history interaction records has reached 20.9471766848816% of the total.
Evaluate   : 100%|███████████████████████████████████████████| 13354/13354 [00:17<00:00, 777.63it/s]


It took 80.70 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.0769), ('mrr@10', 0.1422), ('ndcg@10', 0.0716), ('hit@10', 0.3239), ('precision@10', 0.0422)])}
running RecVAE...


command line args [-f /root/.local/share/jupyter/runtime/kernel-a7e19ae3-fb9e-4161-a147-e3d1f962669a.json] will not be used in RecBole
Max value of user's history interaction records has reached 20.9471766848816% of the total.
Evaluate   : 100%|███████████████████████████████████████████| 13354/13354 [00:57<00:00, 232.16it/s]


It took 10.09 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.0849), ('mrr@10', 0.1697), ('ndcg@10', 0.0828), ('hit@10', 0.3532), ('precision@10', 0.047)])}
running ConvNCF...


command line args [-f /root/.local/share/jupyter/runtime/kernel-a7e19ae3-fb9e-4161-a147-e3d1f962669a.json] will not be used in RecBole
Evaluate   :  49%|█████████████████████▉                       | 6522/13354 [37:10<37:36,  3.03it/s]

In [ ]:
result = run_recbole(model='MultiVAE', dataset = 'recbox_data',config_dict = parameter_dict )